In [29]:
import numpy as np
import random
import gymnasium as gym


In [30]:
env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.1) 

In [31]:
epsilon = 0.1 # tasa de exploración
alpha = 0.1 # tasa de aprendizaje
gamma = 0.99 # factor de descuento
episodios = 1000
max_pasos = 100

In [32]:
def epsilon_greed1y(Q_valores, epsilon):
    if np.random() < epsilon:
        return np.randint(0, len(Q_valores) - 1 )
    else:
        return np.argmax(Q_valores)    

In [33]:
import numpy as np
import random
import gymnasium as gym

# Parámetros de discretización para MountainCar-v0
NUM_BINS_POS = 10
NUM_BINS_VEL = 10

# Crear los bins para cada dimensión (usamos num_bins+1 para incluir los extremos)
bins_pos = np.linspace(-1.2, 0.6, NUM_BINS_POS + 1)
bins_vel = np.linspace(-0.07, 0.07, NUM_BINS_VEL + 1)

def discretize_state(state, bins_pos, bins_vel):
    """
    Convierte un estado continuo [posición, velocidad] en un índice discreto.
    """
    pos, vel = state
    # np.digitize devuelve índices entre 1 y len(bins)
    pos_bin = np.digitize(pos, bins_pos) - 1
    vel_bin = np.digitize(vel, bins_vel) - 1
    # Asegurarse que el índice no exceda el número de bins - 1
    pos_bin = min(pos_bin, NUM_BINS_POS - 1)
    vel_bin = min(vel_bin, NUM_BINS_VEL - 1)
    # Mapear el par (pos_bin, vel_bin) a un único índice
    return pos_bin * NUM_BINS_VEL + vel_bin

def epsilon_greedy(Q_values, epsilon):
    """
    Selecciona una acción según la política epsilon-greedy.
    
    Parámetros:
      Q_values: vector de valores Q para el estado actual.
      epsilon: probabilidad de explorar.
    """
    if random.random() < epsilon:
        return random.randint(0, len(Q_values) - 1)
    else:
        return np.argmax(Q_values)

def dyna_q(env, num_episodes=500, alpha=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    """
    Implementación tabular de Dyna-Q para entornos discretizados.
    
    Parámetros:
      env: entorno Gymnasium.
      num_episodes: número de episodios de entrenamiento.
      alpha: tasa de aprendizaje.
      gamma: factor de descuento.
      epsilon: probabilidad de exploración en la política epsilon-greedy.
      planning_steps: número de actualizaciones de planeación por paso real.
      
    Retorna:
      Q: tabla Q final de forma (n_estados, n_acciones).
    """
    # El total de estados discretos es NUM_BINS_POS * NUM_BINS_VEL
    n_states = NUM_BINS_POS * NUM_BINS_VEL  
    n_actions = env.action_space.n

    # Inicializar Q-table en cero
    Q = np.zeros((n_states, n_actions))
    
    # Inicializar el modelo: model[s][a] = (reward, next_state)
    model = {}
    for s in range(n_states):
        model[s] = {}
        for a in range(n_actions):
            model[s][a] = (0.0, s)  # Valor por defecto

    for episode in range(num_episodes):
        # En Gymnasium el reset devuelve (observacion, info)
        obs, _ = env.reset()
        state = discretize_state(obs, bins_pos, bins_vel)
        done = False

        while not done:
            # Seleccionar acción usando epsilon-greedy
            action = epsilon_greedy(Q[state], epsilon)
            
            # Realizar la acción en el entorno
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            next_state = discretize_state(next_obs, bins_pos, bins_vel)
            
            # Actualizar Q con la experiencia real (regla de Q-learning)
            best_next_action = np.argmax(Q[next_state])
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])
            
            # Actualizar el modelo con la transición observada
            model[state][action] = (reward, next_state)
            
            # Fase de planeación: actualizar Q utilizando transiciones almacenadas
            for _ in range(planning_steps):
                # Seleccionar aleatoriamente un estado y acción que ya se hayan visitado
                sampled_state = random.randint(0, n_states - 1)
                sampled_action = random.randint(0, n_actions - 1)
                r_sampled, s_prime_sampled = model[sampled_state][sampled_action]
                best_a_sampled = np.argmax(Q[s_prime_sampled])
                Q[sampled_state, sampled_action] += alpha * (r_sampled + gamma * Q[s_prime_sampled, best_a_sampled]
                                                             - Q[sampled_state, sampled_action])
            
            # Actualizar el estado
            state = next_state
        
        # (Opcional) Puedes imprimir información del episodio
        print(f"Episode {episode+1}/{num_episodes} finalizado.")

    return Q

if __name__ == "__main__":
    # Crear el entorno MountainCar-v0 de Gymnasium
    env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.1)
    
    # Ejecutar el entrenamiento Dyna-Q
    Q_final = dyna_q(env, num_episodes=500, alpha=0.1, gamma=0.99, epsilon=0.1, planning_steps=10)
    print("Tabla Q entrenada (forma):", Q_final.shape)


Episode 1/500 finalizado.
Episode 2/500 finalizado.
Episode 3/500 finalizado.
Episode 4/500 finalizado.
Episode 5/500 finalizado.
Episode 6/500 finalizado.
Episode 7/500 finalizado.
Episode 8/500 finalizado.
Episode 9/500 finalizado.
Episode 10/500 finalizado.
Episode 11/500 finalizado.
Episode 12/500 finalizado.
Episode 13/500 finalizado.
Episode 14/500 finalizado.
Episode 15/500 finalizado.
Episode 16/500 finalizado.
Episode 17/500 finalizado.
Episode 18/500 finalizado.
Episode 19/500 finalizado.
Episode 20/500 finalizado.
Episode 21/500 finalizado.
Episode 22/500 finalizado.
Episode 23/500 finalizado.
Episode 24/500 finalizado.
Episode 25/500 finalizado.
Episode 26/500 finalizado.
Episode 27/500 finalizado.
Episode 28/500 finalizado.
Episode 29/500 finalizado.
Episode 30/500 finalizado.
Episode 31/500 finalizado.
Episode 32/500 finalizado.
Episode 33/500 finalizado.
Episode 34/500 finalizado.
Episode 35/500 finalizado.


KeyError: -5

In [ ]:
import numpy as np
import random
import gymnasium as gym

# Parámetros de discretización para MountainCar-v0
NUM_BINS_POS = 20
NUM_BINS_VEL = 20

# Crear los bins para cada dimensión (usamos num_bins+1 para incluir los extremos)
bins_pos = np.linspace(-1.2, 0.6, NUM_BINS_POS + 1)
bins_vel = np.linspace(-0.07, 0.07, NUM_BINS_VEL + 1)

def discretize_state(state, bins_pos, bins_vel):
    """
    Convierte un estado continuo [posición, velocidad] en un índice discreto.
    """
    pos, vel = state
    # Obtener el índice y restar 1
    pos_bin = np.digitize(pos, bins_pos) - 1
    vel_bin = np.digitize(vel, bins_vel) - 1
    # Asegurarse de que los índices estén dentro del rango válido usando np.clip
    pos_bin = np.clip(pos_bin, 0, NUM_BINS_POS - 1)
    vel_bin = np.clip(vel_bin, 0, NUM_BINS_VEL - 1)
    # Mapear el par (pos_bin, vel_bin) a un único índice
    return pos_bin * NUM_BINS_VEL + vel_bin

def epsilon_greedy(Q_values, epsilon):
    """
    Selecciona una acción según la política epsilon-greedy.
    
    Parámetros:
      Q_values: vector de valores Q para el estado actual.
      epsilon: probabilidad de explorar.
    """
    if random.random() < epsilon:
        return random.randint(0, len(Q_values) - 1)
    else:
        return np.argmax(Q_values)

def dyna_q(env, num_episodes=500, alpha=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    """
    Implementación tabular de Dyna-Q para entornos discretizados.
    
    Parámetros:
      env: entorno Gymnasium.
      num_episodes: número de episodios de entrenamiento.
      alpha: tasa de aprendizaje.
      gamma: factor de descuento.
      epsilon: probabilidad de exploración en la política epsilon-greedy.
      planning_steps: número de actualizaciones de planeación por paso real.
      
    Retorna:
      Q: tabla Q final de forma (n_estados, n_acciones).
    """
    # El total de estados discretos es NUM_BINS_POS * NUM_BINS_VEL
    n_states = NUM_BINS_POS * NUM_BINS_VEL  
    n_actions = env.action_space.n

    # Inicializar Q-table en cero
    Q = np.zeros((n_states, n_actions))
    
    # Inicializar el modelo: model[s][a] = (reward, next_state)
    model = {}
    for s in range(n_states):
        model[s] = {}
        for a in range(n_actions):
            model[s][a] = (0.0, s)  # Valor por defecto

    for episode in range(num_episodes):
        # En Gymnasium el reset devuelve (observacion, info)
        obs, _ = env.reset()
        state = discretize_state(obs, bins_pos, bins_vel)
        done = False

        while not done:
            # Seleccionar acción usando epsilon-greedy
            action = epsilon_greedy(Q[state], epsilon)
            
            # Realizar la acción en el entorno
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            next_state = discretize_state(next_obs, bins_pos, bins_vel)
            
            # Actualizar Q con la experiencia real (regla de Q-learning)
            best_next_action = np.argmax(Q[next_state])
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])
            
            # Actualizar el modelo con la transición observada
            model[state][action] = (reward, next_state)
            
            # Fase de planeación: actualizar Q utilizando transiciones almacenadas
            for _ in range(planning_steps):
                # Seleccionar aleatoriamente un estado y acción que ya se hayan visitado
                sampled_state = random.randint(0, n_states - 1)
                sampled_action = random.randint(0, n_actions - 1)
                r_sampled, s_prime_sampled = model[sampled_state][sampled_action]
                best_a_sampled = np.argmax(Q[s_prime_sampled])
                Q[sampled_state, sampled_action] += alpha * (r_sampled + gamma * Q[s_prime_sampled, best_a_sampled]
                                                             - Q[sampled_state, sampled_action])
            
            # Actualizar el estado
            state = next_state
        
        # (Opcional) Puedes imprimir información del episodio
        print(f"Episode {episode+1}/{num_episodes} finalizado.")

    return Q

if __name__ == "__main__":
    # Crear el entorno MountainCar-v0 de Gymnasium
    env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.1)
    
    # Ejecutar el entrenamiento Dyna-Q
    Q_final = dyna_q(env, num_episodes=500, alpha=0.1, gamma=0.99, epsilon=0.1, planning_steps=10)
    print("Tabla Q entrenada (forma):", Q_final.shape)


In [ ]:
import numpy as np
import random
import gymnasium as gym
import matplotlib.pyplot as plt
import pandas as pd

# Parámetros de discretización para MountainCar-v0
NUM_BINS_POS = 20
NUM_BINS_VEL = 20

# Crear los bins para cada dimensión (se usa num_bins+1 para incluir los extremos)
bins_pos = np.linspace(-1.2, 0.6, NUM_BINS_POS + 1)
bins_vel = np.linspace(-0.07, 0.07, NUM_BINS_VEL + 1)

def discretize_state(state, bins_pos, bins_vel):
    """
    Convierte un estado continuo [posición, velocidad] en un índice discreto.
    """
    pos, vel = state
    # Usamos np.clip para asegurarnos que el índice esté dentro de [0, num_bins-1]
    pos_bin = np.clip(np.digitize(pos, bins_pos) - 1, 0, NUM_BINS_POS - 1)
    vel_bin = np.clip(np.digitize(vel, bins_vel) - 1, 0, NUM_BINS_VEL - 1)
    return pos_bin * NUM_BINS_VEL + vel_bin

def epsilon_greedy(Q_values, epsilon):
    """
    Selecciona una acción según la política epsilon-greedy.
    
    Parámetros:
      Q_values: vector de valores Q para el estado actual.
      epsilon: probabilidad de explorar.
    """
    if random.random() < epsilon:
        return random.randint(0, len(Q_values) - 1)
    else:
        return np.argmax(Q_values)

def dyna_q_train(env, num_episodes=500, alpha=0.1, gamma=0.99, epsilon=0.1, planning_steps=10):
    """
    Entrena Dyna-Q en un entorno discretizado.
    
    Retorna:
      Q: la tabla Q aprendida.
      training_rewards: lista con la recompensa total obtenida en cada episodio.
    """
    n_states = NUM_BINS_POS * NUM_BINS_VEL  
    n_actions = env.action_space.n

    Q = np.zeros((n_states, n_actions))
    # Inicializar el modelo: model[s][a] = (reward, next_state)
    model = {s: {a: (0.0, s) for a in range(n_actions)} for s in range(n_states)}
    
    training_rewards = []  # Recompensa total por episodio
    
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = discretize_state(obs, bins_pos, bins_vel)
        done = False
        total_reward = 0.0
        
        while not done:
            action = epsilon_greedy(Q[state], epsilon)
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            next_state = discretize_state(next_obs, bins_pos, bins_vel)
            total_reward += reward
            
            # Actualización de Q (Q-learning)
            best_next_action = np.argmax(Q[next_state])
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])
            
            # Actualizar el modelo con la transición observada
            model[state][action] = (reward, next_state)
            
            # Fase de planeación: actualizar Q utilizando transiciones almacenadas
            for _ in range(planning_steps):
                sampled_state = random.randint(0, n_states - 1)
                sampled_action = random.randint(0, n_actions - 1)
                r_sampled, s_prime_sampled = model[sampled_state][sampled_action]
                best_a_sampled = np.argmax(Q[s_prime_sampled])
                Q[sampled_state, sampled_action] += alpha * (r_sampled + gamma * Q[s_prime_sampled, best_a_sampled]
                                                             - Q[sampled_state, sampled_action])
                
            state = next_state
        
        training_rewards.append(total_reward)
        print(f"Episode {episode+1}/{num_episodes} terminado. Recompensa: {total_reward}")
        
    return Q, training_rewards

def evaluate_policy(env, Q, render=False):
    """
    Evalúa la política resultante (siempre eligiendo la acción greedy)
    y retorna la recompensa total obtenida en un episodio de prueba.
    """
    obs, _ = env.reset()
    state = discretize_state(obs, bins_pos, bins_vel)
    done = False
    total_reward = 0.0
    while not done:
        action = np.argmax(Q[state])  # Política greedy
        next_obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward
        state = discretize_state(next_obs, bins_pos, bins_vel)
        if render:
            env.render()
    return total_reward

# Parámetros para los experimentos
num_experimentos = 10
num_episodes = 500

# Para almacenar los resultados de cada experimento
all_training_rewards = []  # Lista de listas, una por experimento
evaluation_rewards = []    # Recompensa de evaluación (política greedy) por experimento

# Ejecutar 10 experimentos
for exp in range(num_experimentos):
    print(f"\n=== Experimento {exp+1} ===")
    # Crear un nuevo entorno para cada experimento
    env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.5)
    Q, training_rewards = dyna_q_train(env, num_episodes=num_episodes, alpha=0.01, gamma=0.99, epsilon=0.4, planning_steps=10)
    all_training_rewards.append(training_rewards)
    # Evaluar la política resultante
    eval_reward = evaluate_policy(env, Q)
    evaluation_rewards.append(eval_reward)
    env.close()

# Procesar las recompensas de entrenamiento para graficar
all_training_rewards = np.array(all_training_rewards)  # Dimensión: (num_experimentos, num_episodes)
mean_training_rewards = np.mean(all_training_rewards, axis=0)
std_training_rewards = np.std(all_training_rewards, axis=0)

# Graficar la recompensa promedio durante el entrenamiento
plt.figure(figsize=(10, 5))
episodes = np.arange(1, num_episodes + 1)
plt.plot(episodes, mean_training_rewards, label="Recompensa Promedio")
plt.fill_between(episodes, mean_training_rewards - std_training_rewards, mean_training_rewards + std_training_rewards,
                 alpha=0.2, label="±1 Desv. Estándar")
plt.xlabel("Episodios")
plt.ylabel("Recompensa Total")
plt.title("Curva de Recompensa durante el Entrenamiento (Dyna-Q en MountainCar)")
plt.legend()
plt.show()

# Crear una tabla con los resultados de evaluación
df = pd.DataFrame({
    "Experimento": np.arange(1, num_experimentos + 1),
    "Recompensa Evaluación": evaluation_rewards
})
mean_eval = np.mean(evaluation_rewards)
std_eval = np.std(evaluation_rewards)
summary_df = pd.DataFrame({
    "Media Recompensa": [mean_eval],
    "Desviación Estándar": [std_eval]
})

print("Resultados de Evaluación por Experimento:")
print(df)
print("\nResumen (Media y Desviación Estándar) de la Recompensa Evaluación:")
print(summary_df)



=== Experimento 1 ===
Episode 1/500 terminado. Recompensa: -200.0
Episode 2/500 terminado. Recompensa: -200.0
Episode 3/500 terminado. Recompensa: -200.0
Episode 4/500 terminado. Recompensa: -200.0
Episode 5/500 terminado. Recompensa: -200.0
Episode 6/500 terminado. Recompensa: -200.0
Episode 7/500 terminado. Recompensa: -200.0
Episode 8/500 terminado. Recompensa: -200.0
Episode 9/500 terminado. Recompensa: -200.0
Episode 10/500 terminado. Recompensa: -200.0
Episode 11/500 terminado. Recompensa: -200.0
Episode 12/500 terminado. Recompensa: -200.0
Episode 13/500 terminado. Recompensa: -200.0
Episode 14/500 terminado. Recompensa: -200.0
Episode 15/500 terminado. Recompensa: -200.0
Episode 16/500 terminado. Recompensa: -200.0
Episode 17/500 terminado. Recompensa: -200.0
Episode 18/500 terminado. Recompensa: -200.0
Episode 19/500 terminado. Recompensa: -200.0
Episode 20/500 terminado. Recompensa: -200.0
Episode 21/500 terminado. Recompensa: -200.0
Episode 22/500 terminado. Recompensa: -20

KeyboardInterrupt: 

In [ ]:
import numpy as np
import random
import gymnasium as gym
import matplotlib.pyplot as plt
import pandas as pd

# Parámetros de discretización (ajustables)
NUM_BINS_POS = 20    # Puedes aumentar a 30 o más para mayor precisión
NUM_BINS_VEL = 20

# Crear los bins para cada dimensión (se usa num_bins+1 para incluir los extremos)
bins_pos = np.linspace(-1.2, 0.6, NUM_BINS_POS + 1)
bins_vel = np.linspace(-0.07, 0.07, NUM_BINS_VEL + 1)

def discretize_state(state, bins_pos, bins_vel):
    """
    Convierte un estado continuo [posición, velocidad] en un índice discreto.
    """
    pos, vel = state
    pos_bin = np.clip(np.digitize(pos, bins_pos) - 1, 0, NUM_BINS_POS - 1)
    vel_bin = np.clip(np.digitize(vel, bins_vel) - 1, 0, NUM_BINS_VEL - 1)
    return pos_bin * NUM_BINS_VEL + vel_bin

def epsilon_greedy(Q_values, epsilon):
    """
    Selecciona una acción según la política epsilon-greedy.
    """
    if random.random() < epsilon:
        return random.randint(0, len(Q_values) - 1)
    else:
        return np.argmax(Q_values)

def dyna_q_train(env, num_episodes=500, alpha=0.1, gamma=0.99, epsilon=1.0, planning_steps=20):
    """
    Entrena Dyna-Q en un entorno discretizado.
    Retorna:
      Q: la tabla Q aprendida.
      training_rewards: lista con la recompensa total obtenida en cada episodio.
    """
    n_states = NUM_BINS_POS * NUM_BINS_VEL  
    n_actions = env.action_space.n

    Q = np.zeros((n_states, n_actions))
    model = {s: {a: (0.0, s) for a in range(n_actions)} for s in range(n_states)}
    
    training_rewards = []  # Recompensa total por episodio
    
    # Parámetros para decaimiento de epsilon
    epsilon_min = 0.05
    epsilon_decay = 0.995
    
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = discretize_state(obs, bins_pos, bins_vel)
        done = False
        total_reward = 0.0
        
        while not done:
            action = epsilon_greedy(Q[state], epsilon)
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            next_state = discretize_state(next_obs, bins_pos, bins_vel)
            total_reward += reward
            
            best_next_action = np.argmax(Q[next_state])
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])
            
            model[state][action] = (reward, next_state)
            
            for _ in range(planning_steps):
                sampled_state = random.randint(0, n_states - 1)
                sampled_action = random.randint(0, n_actions - 1)
                r_sampled, s_prime_sampled = model[sampled_state][sampled_action]
                best_a_sampled = np.argmax(Q[s_prime_sampled])
                Q[sampled_state, sampled_action] += alpha * (r_sampled + gamma * Q[s_prime_sampled, best_a_sampled]
                                                             - Q[sampled_state, sampled_action])
                
            state = next_state
        
        training_rewards.append(total_reward)
        # Actualizar epsilon (decaimiento)
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        print(f"Episode {episode+1}/{num_episodes} terminado. Recompensa: {total_reward}, epsilon: {epsilon:.3f}")
        
    return Q, training_rewards

def evaluate_policy(env, Q, render=False):
    """
    Evalúa la política resultante (acción greedy) y retorna la recompensa total.
    """
    obs, _ = env.reset()
    state = discretize_state(obs, bins_pos, bins_vel)
    done = False
    total_reward = 0.0
    while not done:
        action = np.argmax(Q[state])  # Política greedy
        next_obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward
        state = discretize_state(next_obs, bins_pos, bins_vel)
        if render:
            env.render()
    return total_reward

# Parámetros para los experimentos
num_experimentos = 10
num_episodes = 500

all_training_rewards = []
evaluation_rewards = []

for exp in range(num_experimentos):
    print(f"\n=== Experimento {exp+1} ===")
    env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.1)
    Q, training_rewards = dyna_q_train(env, num_episodes=num_episodes, alpha=0.8, gamma=0.8, epsilon=1.0, planning_steps=25)
    all_training_rewards.append(training_rewards)
    eval_reward = evaluate_policy(env, Q)
    evaluation_rewards.append(eval_reward)
    env.close()

all_training_rewards = np.array(all_training_rewards)
mean_training_rewards = np.mean(all_training_rewards, axis=0)
std_training_rewards = np.std(all_training_rewards, axis=0)

plt.figure(figsize=(10, 5))
episodes = np.arange(1, num_episodes + 1)
plt.plot(episodes, mean_training_rewards, label="Recompensa Promedio")
plt.fill_between(episodes, mean_training_rewards - std_training_rewards, mean_training_rewards + std_training_rewards,
                 alpha=0.2, label="±1 Desv. Estándar")
plt.xlabel("Episodios")
plt.ylabel("Recompensa Total")
plt.title("Curva de Recompensa durante el Entrenamiento (Dyna-Q en MountainCar)")
plt.legend()
plt.show()

df = pd.DataFrame({
    "Experimento": np.arange(1, num_experimentos + 1),
    "Recompensa Evaluación": evaluation_rewards
})
mean_eval = np.mean(evaluation_rewards)
std_eval = np.std(evaluation_rewards)
summary_df = pd.DataFrame({
    "Media Recompensa": [mean_eval],
    "Desviación Estándar": [std_eval]
})

print("Resultados de Evaluación por Experimento:")
print(df)
print("\nResumen (Media y Desviación Estándar) de la Recompensa Evaluación:")
print(summary_df)



=== Experimento 1 ===
Episode 1/500 terminado. Recompensa: -200.0, epsilon: 0.995
Episode 2/500 terminado. Recompensa: -200.0, epsilon: 0.990
Episode 3/500 terminado. Recompensa: -200.0, epsilon: 0.985
Episode 4/500 terminado. Recompensa: -200.0, epsilon: 0.980
Episode 5/500 terminado. Recompensa: -200.0, epsilon: 0.975
Episode 6/500 terminado. Recompensa: -200.0, epsilon: 0.970
Episode 7/500 terminado. Recompensa: -200.0, epsilon: 0.966
Episode 8/500 terminado. Recompensa: -200.0, epsilon: 0.961
Episode 9/500 terminado. Recompensa: -200.0, epsilon: 0.956
Episode 10/500 terminado. Recompensa: -200.0, epsilon: 0.951
Episode 11/500 terminado. Recompensa: -200.0, epsilon: 0.946
Episode 12/500 terminado. Recompensa: -200.0, epsilon: 0.942
Episode 13/500 terminado. Recompensa: -200.0, epsilon: 0.937
Episode 14/500 terminado. Recompensa: -200.0, epsilon: 0.932
Episode 15/500 terminado. Recompensa: -200.0, epsilon: 0.928
Episode 16/500 terminado. Recompensa: -200.0, epsilon: 0.923
Episode 17